In [ ]:
import pydicom

In [ ]:
pydicom.dcmread("./files/2.25.130320686061845719052479996342451170932-6-1-qr249j.dcm")

In [ ]:
import os
import numpy as np
import pydicom
import pydicom_seg
import SimpleITK as sitk
nifti_mask = "./prostate_mask.nii.gz"
reference_dicoms = "./files"
output_dcmseg = "./pros_mask.dcm"

segmentation: sitk.Image = sitk.ReadImage(nifti_mask)
segmentation = sitk.Cast(segmentation, sitk.sitkUInt8) 
print(np.unique(sitk.GetArrayFromImage(segmentation)))
dicom_series_paths = [os.path.join(reference_dicoms, f) for f in os.listdir(reference_dicoms) if f.endswith('.dcm')]
source_images = [
    pydicom.dcmread(x, stop_before_pixels=True)
    for x in dicom_series_paths
]

# fill missing ImagePositionPatient 
for source_image in source_images:
    if 'ImagePositionPatient' not in source_image:
        source_image.ImagePositionPatient = [0, 0, 0]

template = pydicom_seg.template.from_dcmqi_metainfo('test.json')
writer = pydicom_seg.MultiClassWriter(
    template=template,
    inplane_cropping=False,
    skip_empty_slices=False,
    skip_missing_segment=False
)

dcm = writer.write(segmentation, source_images)
dcm.SegmentSequence[0].SegmentLabel = "Prostate Mask"
dcm.PatientComments = "Project:PROSTATE Subject:101 Session:101_20141111_mr"
dcm.save_as('out/pros_mask.dcm')